In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import all_estimators
from collections import defaultdict
from shap_ultils import get_shap, get_shap_interactions
from sklearn.ensemble import HistGradientBoostingClassifier

# Load dataset
df = pd.read_csv('1_Reults.20887.interaction_terms_go_family_topfam.tsv',sep='\t')
X = df.iloc[:,1:-2]
y = list(df['protein_family'])

d = defaultdict(list)
# Data sets
l = list()

fin = open('list.txt','r')
for f in fin:
    l.append(f.strip())
fin.close()
l = [col.strip() for col in l]
colunas_validas = [col for col in l if col in X.columns]


# All Features
d['All Features'] = [X[colunas_validas[0:]],y]

## Top 100 
#d['Top 100'] = [X[colunas_validas[0:100]],y]

# Top 200 
#d['Top 200'] = [X[colunas_validas[0:200]],y]

# Top 300 
#d['Top 300'] = [X[colunas_validas[0:300]],y]

# Top 400 
#d['Top 400'] = [X[colunas_validas[0:400]],y]

# Top 500 
#d['Top 500'] = [X[colunas_validas[0:500]],y]

# interatomic interactions datasets
#d['Interatomic interactions'] = [X.iloc[:,0:12],y]

# Chemical properties of amino acid side chains (CPAASC) 
#d['Chemical properties of amino acid side chains (CPAASC)'] = [X.iloc[:,13:20],y]

# Monopeptide 
#d['Monopeptide'] = [X.iloc[:,21:46],y]

# Dipeptide 
#d['Dipeptide'] = [X.iloc[:,21:722],y]

# Tripeptide 
#d['Tripeptide'] = [X.iloc[:,723:],y]

classifiers = [est for est in all_estimators(type_filter='classifier')]
#group = ['BaggingClassifier','ExtraTreesClassifier','GradientBoostingClassifier','HistGradientBoostingClassifier','RandomForestClassifier']
group = ['HistGradientBoostingClassifier']

classes = np.unique(np.sort(y))

for x in d:
    print(x)
    X_train, X_test, y_train, y_test = train_test_split(d[x][0], d[x][1],test_size=0.20)
    column_names = X_train.columns
    feature_labels = column_names.tolist()

    # Train on full training set
    print ('Train on full training set ...')
    model = HistGradientBoostingClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print ('Explain your model ...')
    for class_i, class_label in enumerate(classes):
        print(class_i, class_label)
        class_label = x.replace(' ',' ')+" "+class_label.replace(' ', ' ')
        class_label = class_label.replace('/', ' ')
        #using permutation explainer because shap does not recognize HistGradientBoostingClassifier as a tree based model in TreeExplainer
        my_explainer = get_shap(model.predict_proba, X_test, feature_labels, class_label, "", multiclass_index=class_i)
        # Unfortunately our only solution now is to use permutation explainer
        # since shap interactions only supports TreeExplainers, we are not able to generate shap interaction scores for the permutation explainer
        #interaction_df = get_shap_interactions(model.predict_proba, X_test, feature_labels, "my response variable", "my example plot title", X_train=X_train, cor_threshold=0.005)

ModuleNotFoundError: No module named 'xgboost'